# Calculate the $\Delta G$ between two residues


Let A and B are the two residues in a chemical reaction then,

$AH + B ⇌ A^{-} + BH$
 
Free energy
 
$\Delta G = - RT \ln\frac{[A^{-}] [BH]} {[AH]  [B]}$
 
But for microstate since A and B are need to appear in each microstate then
 

 
 $\Delta G = - RT \ln\frac{[A^{-}| BH ]} {[AH | B]}$
 
where RT =  1.36 in log10 and  RT = 0.593  in ln 





#  The will also find entropy term from Gibs free energy:

$\Delta G  = \Delta H - T \Delta S $ 

$T \Delta S   = \Delta H - \Delta G $ 

In [1]:
import ms_analysis as msa

In [2]:
mc = msa.MSout("../ms_out/pH5eH0ms.txt")

In [3]:
ms_orig_lst = [[ms.E, ms.count, ms.state] for  ms in list((mc.microstates.values()))]
ms_orig_lst = sorted(ms_orig_lst, key = lambda x:x[0])

In [4]:
import numpy as np
#res1 = ["GLUA35", 0]
#res2 = ["ASPA52", -1]
def Count_residue_condition(res1, res2, ms_list):
    # here we create the separate dictionary for list beacuase we need to track later which conformers is
    # from which residues
    res1_iconf = {conf.iconf: int(conf.crg) for conf in msa.conformers \
                  if (conf.resid[:4] + str(int(conf.resid[4:8]))) in res1[0]}

    res2_iconf = {conf.iconf: int(conf.crg) for conf in msa.conformers \
                  if (conf.resid[:4] + str(int(conf.resid[4:8]))) in res2[0]}
   
    combine_iconf = {**res1_iconf, **res2_iconf}
    # this is for look up in order to track whether given residues are in fixed ms residues list or not.
    # If your have given the charge that is not present in microstate or in fixed conformers it will check.
    check_crg = True
    for i, j in combine_iconf.items():
        if i in mc.fixed_iconfs:
            if i in res1_iconf.keys():
                print(f"{res1[0]} is fixed with charge {res1[1]}")
                if res1[1] != j:
                    check_crg = False
                    print(f"Stop! {res1[0]} not found in {res1[1]} charge state")
                    return "No Count", "No ms energy"
            if i in res2_iconf.keys():
                print(f"{res2[0]} is fixed with charge {res2[1]}")
                if res2[1] != j:
                    check_crg = False
                    print(f"Stop! {res2[0]} not found in {res2[1]} charge state")
                    return "Given Charge combination doesn't exit", "Given Charge combination doesn't exit"


     #now find the confomers position in ms state in order to speed up searching position. 
    #take first ms state for this.
    #first_ms_lookup = list(mc.microstates.values())[0].state
    first_ms_lookup = ms_list[0][2]
    res1_position_ms, res2_position_ms = -1, -1
    for i in range(len(first_ms_lookup)):
        if first_ms_lookup[i] in res1_iconf:
            res1_position_ms = i
        if first_ms_lookup[i] in res2_iconf:
            res2_position_ms = i
    

    # # group reisudes based on given condition. Make list of the conformers for look up.
    res1_iconf_given_crg = [x for x, y in res1_iconf.items() if y == res1[1]]
    res2_iconf_given_crg = [x for x, y in res2_iconf.items() if y == res2[1]]
    


    # # count based given charge condition
    count_given_condition = 0
    # # this will find all the sum of ms state to get count

    # if any of the residues are fixed then we willn't see the that residues in ms state
    energy_sum_condition= 0
    if check_crg:
        for i in ms_list:
            if res1_position_ms != -1 and res2_position_ms != -1:
                if i[2][res1_position_ms] in res1_iconf_given_crg and i[2][res2_position_ms] in res2_iconf_given_crg:
                    count_given_condition += i[1]
                    energy_sum_condition += i[1] * i[0]
            elif res1_position_ms != -1 and res2_position_ms == -1:
                if i[2][res1_position_ms] in res1_iconf_given_crg:
                    count_given_condition += i[1]
                    energy_sum_condition += i[1] * i[0]
                # check if that charge state appear in ms
            elif res1_position_ms == -1 and res2_position_ms != -1:
                if i[2][res2_position_ms] in res2_iconf_given_crg:
                    count_given_condition += i[1]
                    energy_sum_condition += i[1] * i[0]
            else:
                count_given_condition += i[1]
                energy_sum_condition += i[1] * i[0]
        if count_given_condition == 0:
             return "Given Charge combination doesn't exit", "Given Charge combination doesn't exit"
        else:
            return count_given_condition, round((energy_sum_condition / count_given_condition), 3)


In [5]:
import math
# RT =  1.36 #1.36 if log10 and if RT = 0.593 if in ln 
def deltaG_from_count(reactant, product):
    # if residues are fixed then left or right will not return int
    if type(reactant) != int or type(product) != int:
        return ("Residues choosen are fixed")
    elif reactant == 0 or  product == 0:
        return ("Value Error")
    else:
        deltaG = round(- 1.36 * math.log10(product /reactant), 3)
        return deltaG
   

In [6]:
# Glu-35 has experimental pKa 6.1 whereas Asp-52 has 3.4
# Glu-35 is proton donor and ASP52 is acceptor.
# reaction: GLU35(0) + ASP52(-1) == GLU35(-1) + ASP52(0)
# chisquare: GLU35 = 0.573, ASP52 = 0.094
# slope: GLU35 = 0.954, ASP52 = 0.940
# 0 -1 >> -1 0
reactant = Count_residue_condition(["GLUA35", 0], ["ASPA52", -1], ms_orig_lst)
product=  Count_residue_condition(["GLUA35", -1], ["ASPA52", 0],  ms_orig_lst)
print(f"Average reactant ms energy: {reactant[1]} Kcal/Mol")
print(f"Average product ms energy: {product[1]} Kcal/Mol")
print(f"Reactant number: {reactant[0]}")
print(f"Product number: {product[0]}")
print(f"Total reactant and product: {reactant[0] + product[0]}")
print(f"Total all count: {mc.N_ms}")
print(f"DeltaG: {deltaG_from_count(reactant[0], product[0])} Kcal/Mol")
print(f"DeltaH: {product[1] - reactant[1]:.3f} Kcal/Mol")
print(f"T*DeltaS: {(product[1] - reactant[1]) - deltaG_from_count(reactant[0], product[0]):.3f} Kcal/Mol")


Average reactant ms energy: -198.698 Kcal/Mol
Average product ms energy: -196.105 Kcal/Mol
Reactant number: 817457
Product number: 24832
Total reactant and product: 842289
Total all count: 1500000
DeltaG: 2.064 Kcal/Mol
DeltaH: 2.593 Kcal/Mol
T*DeltaS: 0.529 Kcal/Mol


In [7]:
# energy for thermodynamic box:
## 0 0 >>>  0 -1
r1 = Count_residue_condition(["GLUA35", 0], ["ASPA52", 0], ms_orig_lst)
p1 = Count_residue_condition(["GLUA35", 0], ["ASPA52", -1], ms_orig_lst)
print(f"Average reactant ms energy: {r1[1]} Kcal/Mol")
print(f"Average product ms energy: {p1[1]} Kcal/Mol")
print(f"Reactant number: {r1[0]}")
print(f"Product number: {p1[0]}")
print(f"Total reactant and product: {r1[0]+ p1[0]}")
print(f"Total all count: {mc.N_ms}")
print(f"DeltaG: {deltaG_from_count(r1[0], p1[0])} Kcal/Mol")
print(f"DeltaH: {p1[1] - r1[1]:.3f} Kcal/Mol")
print(f"T*DeltaS: {(p1[1] - r1[1]) - deltaG_from_count(r1[0], p1[0]):.3f} Kcal/Mol")


Average reactant ms energy: -195.017 Kcal/Mol
Average product ms energy: -198.698 Kcal/Mol
Reactant number: 5281
Product number: 817457
Total reactant and product: 822738
Total all count: 1500000
DeltaG: -2.978 Kcal/Mol
DeltaH: -3.681 Kcal/Mol
T*DeltaS: -0.703 Kcal/Mol


In [8]:
# energy for thermodynamic box:
## 0 -1 >>>  -1 -1
r2 = Count_residue_condition(["GLUA35", 0], ["ASPA52", -1], ms_orig_lst)
p2 = Count_residue_condition(["GLUA35", -1], ["ASPA52", -1], ms_orig_lst)
print(f"Average reactant ms energy: {r2[1]} Kcal/Mol")
print(f"Average product ms energy: {p2[1]} Kcal/Mol")
print(f"Reactant number: {r2[0]}")
print(f"Product number: {p2[0]}")
print(f"Total reactant and product: {r2[0]+ p2[0]}")
print(f"Total all count: {mc.N_ms}")
print(f"DeltaG: {deltaG_from_count(r2[0], p2[0])} Kcal/Mol")
print(f"DeltaH: {p2[1] - r2[1]:.3f} Kcal/Mol")
print(f"T*DeltaS: {(p2[1] - r2[1]) - deltaG_from_count(r2[0], p2[0]):.3f} Kcal/Mol")

Average reactant ms energy: -198.698 Kcal/Mol
Average product ms energy: -198.906 Kcal/Mol
Reactant number: 817457
Product number: 652430
Total reactant and product: 1469887
Total all count: 1500000
DeltaG: 0.133 Kcal/Mol
DeltaH: -0.208 Kcal/Mol
T*DeltaS: -0.341 Kcal/Mol


In [9]:
# energy for thermodynamic box:
## -1 -1 >>>  -1 0
r3 = Count_residue_condition(["GLUA35", -1], ["ASPA52", -1], ms_orig_lst)
p3 = Count_residue_condition(["GLUA35", -1], ["ASPA52", 0], ms_orig_lst)
print(f"Average reactant ms energy: {r3[1]} Kcal/Mol")
print(f"Average product ms energy: {p3[1]} Kcal/Mol")
print(f"Reactant number: {r3[0]}")
print(f"Product number: {p3[0]}")
print(f"Total reactant and product: {r3[0]+ p3[0]}")
print(f"Total all count: {mc.N_ms}")
print(f"DeltaG: {deltaG_from_count(r3[0], p3[0])} Kcal/Mol")
print(f"DeltaH: {p3[1] - r3[1]:.3f} Kcal/Mol")
print(f"T*DeltaS: {(p3[1] - r3[1]) - deltaG_from_count(r3[0], p3[0]):.3f} Kcal/Mol")


Average reactant ms energy: -198.906 Kcal/Mol
Average product ms energy: -196.105 Kcal/Mol
Reactant number: 652430
Product number: 24832
Total reactant and product: 677262
Total all count: 1500000
DeltaG: 1.931 Kcal/Mol
DeltaH: 2.801 Kcal/Mol
T*DeltaS: 0.870 Kcal/Mol


In [10]:
# energy for thermodynamic box:
## -1 0 >>>  0 0
r4 = Count_residue_condition(["GLUA35", -1], ["ASPA52", 0], ms_orig_lst)
p4 = Count_residue_condition(["GLUA35", 0], ["ASPA52", 0], ms_orig_lst)
print(f"Average reactant ms energy: {r4[1]} Kcal/Mol")
print(f"Average product ms energy: {p4[1]} Kcal/Mol")
print(f"Reactant number: {r4[0]}")
print(f"Product number: {p4[0]}")
print(f"Total reactant and product: {r4[0]+ p4[0]}")
print(f"Total all count: {mc.N_ms}")
print(f"DeltaG: {deltaG_from_count(r4[0], p4[0])} Kcal/Mol")
print(f"DeltaH: {p4[1] - r4[1]:.3f} Kcal/Mol")
print(f"T*DeltaS: {(p4[1] - r4[1]) - deltaG_from_count(r4[0], p4[0]):.3f} Kcal/Mol")

Average reactant ms energy: -196.105 Kcal/Mol
Average product ms energy: -195.017 Kcal/Mol
Reactant number: 24832
Product number: 5281
Total reactant and product: 30113
Total all count: 1500000
DeltaG: 0.914 Kcal/Mol
DeltaH: 1.088 Kcal/Mol
T*DeltaS: 0.174 Kcal/Mol


In [11]:
# energy for thermodynamic box:
## 0 0 >>>  -1 -1
r5 = Count_residue_condition(["GLUA35", 0], ["ASPA52", 0], ms_orig_lst)
p5 = Count_residue_condition(["GLUA35", -1], ["ASPA52", -1], ms_orig_lst)
print(f"Average reactant ms energy: {r5[1]} Kcal/Mol")
print(f"Average product ms energy: {p5[1]} Kcal/Mol")
print(f"Reactant number: {r5[0]}")
print(f"Product number: {p5[0]}")
print(f"Total reactant and product: {r5[0]+ p5[0]}")
print(f"Total all count: {mc.N_ms}")
print(f"DeltaG: {deltaG_from_count(r5[0], p5[0])} Kcal/Mol")
print(f"DeltaH: {p5[1] - r5[1]:.3f} Kcal/Mol")
print(f"T*DeltaS: {(p5[1] - r5[1]) - deltaG_from_count(r5[0], p5[0]):.3f} Kcal/Mol")

Average reactant ms energy: -195.017 Kcal/Mol
Average product ms energy: -198.906 Kcal/Mol
Reactant number: 5281
Product number: 652430
Total reactant and product: 657711
Total all count: 1500000
DeltaG: -2.845 Kcal/Mol
DeltaH: -3.889 Kcal/Mol
T*DeltaS: -1.044 Kcal/Mol
